### Cal-CRAI metric calculation: soil quality
* Mean evaporative stress index
* Mean surface soil moisture anomaly (depending on structure)

In [1]:
import pandas as pd
import os
import sys
import math
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point

# suppress pandas purely educational warnings
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.file_helpers import pull_csv_from_directory, upload_csv_aws, filter_counties
from scripts.utils.write_metadata import append_metadata

### Metric 1
Mean evaporative stress index

In [2]:
evap_stress_link = 's3://ca-climate-index/2b_reproject/natural_systems/agricultural_productivity_conservation/usda_nasa/evaporative_stress_ca_reproject.zarr/'

In [ ]:
ds = xr.open_zarr(evap_stress_link)

# grab variable and load into memory
esi = ds['esi']
esi = esi.compute()
esi

In [ ]:
# determine how many values are extreme negative values -- are these outliers and can be safely removed?
esi.min().values, esi.max().values
# range is -25 to +5

# how many values are below ~ -5? (determining # of outliers)
outliers = (esi < -5).sum()
outliers

# 862 values below -5 (1 value below -25), keep!

### Calculate metric
* mean evaporative stress index

In [ ]:
# check difference between time mean and annual mean -- no difference
esi_ann = esi.groupby('time.year').mean()
esi_ann_avg = esi_ann.mean(dim='year')
esi_ann_avg.min().values, esi_ann_avg.max().values

### Reproject data to census tract projection

In [6]:
df = esi_ann_avg.to_dataframe().reset_index()

In [ ]:
unique_esi_entries = df['esi'].nunique()
print(f'There are {unique_esi_entries} unique entries in the esi column.')

In [ ]:
# Create geometry column from coordinates
geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]

# Create GeoDataFrame
esi_gdf = gpd.GeoDataFrame(df, geometry=geometry)
esi_gdf

In [ ]:
esi_gdf.plot(column='esi', markersize=0.25, cmap='bwr')

In [ ]:
# read in CA census tiger file
census_shp_dir = "s3://ca-climate-index/0_map_data/2021_tiger_census_tract/2021_ca_tract/"
ca_boundaries = gpd.read_file(census_shp_dir)
print("ca_boundaries CRS:", ca_boundaries.crs)

In [ ]:
# Check the current CRS of the GeoDataFrame
print("gdf CRS:", esi_gdf.crs)
esi_gdf.set_crs(epsg="4269", inplace=True)

# If they have different CRS, transform one to match the other
if esi_gdf.crs != ca_boundaries.crs:
    esi_gdf = esi_gdf.to_crs(ca_boundaries.crs)

# Now both GeoDataFrames should have the same CRS
print("Transformed gdf CRS:", esi_gdf.crs)

In [ ]:
# intersect points within geometry -- warning popping up about CRS -- ignore since they are in the same projection
intersect_df = gpd.sjoin_nearest(esi_gdf, ca_boundaries, how='right')
intersect_df_avg = intersect_df.groupby('GEOID')['esi'].agg(['mean']).reset_index()
intersect_df_avg = pd.merge(intersect_df_avg, ca_boundaries, on='GEOID', how='right')
intersect_df_avg = gpd.GeoDataFrame(intersect_df_avg, geometry='geometry')

# reanme column to be interpretable
intersect_df_avg['esi_mean'] = intersect_df_avg['mean']
intersect_df_avg.plot(column='esi_mean')

In [ ]:
# drop unneeded columns
intersect_df_avg = intersect_df_avg[['GEOID', 'esi_mean']]
intersect_df_avg

In [19]:
# save as a csv
intersect_df_avg.to_csv('natural_crop_condition_esi_metric.csv', index=False)

### Metric 2
* Mean surface soil moisture anomaly

In [16]:
soil_anom_link = 's3://ca-climate-index/2b_reproject/natural_systems/agricultural_productivity_conservation/usda_nasa/soil_moisture_anom_ca_reproject.zarr/'

In [ ]:
ds = xr.open_zarr(soil_anom_link)

# grab variable and load into memory
ssma = ds['ssma']
ssma = ssma.compute()
ssma

In [ ]:
ssma.min().values, ssma.max().values

### Calculate metric

In [ ]:
ssma_ann = ssma.groupby('time.year').mean()
ssma_ann_avg = ssma_ann.mean(dim='year')
ssma_ann_avg.min().values, ssma_ann_avg.max().values

### Reproject to census tracts

In [20]:
df = ssma_ann_avg.to_dataframe().reset_index()

In [ ]:
# Create geometry column from coordinates
geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]

# Create GeoDataFrame
ssma_gdf = gpd.GeoDataFrame(df, geometry=geometry)
ssma_gdf

In [ ]:
ssma_gdf.plot(column='ssma', markersize=7, cmap='bwr')
# noting that there are actual nan values for some locations

In [ ]:
# Check the current CRS of the GeoDataFrame
print("gdf CRS:", ssma_gdf.crs)
ssma_gdf.set_crs(epsg="4269", inplace=True)

# If they have different CRS, transform one to match the other
if ssma_gdf.crs != ca_boundaries.crs:
    ssma_gdf = ssma_gdf.to_crs(ca_boundaries.crs)

# Now both GeoDataFrames should have the same CRS
print("Transformed gdf CRS:", ssma_gdf.crs)

In [ ]:
# intersect points within geometry -- warning popping up about CRS -- ignore since they are in the same projection
intersect_df = gpd.sjoin_nearest(ssma_gdf, ca_boundaries, how='right')
intersect_ssma_avg = intersect_df.groupby('GEOID')['ssma'].agg(['mean']).reset_index()
intersect_ssma_avg = pd.merge(intersect_ssma_avg, ca_boundaries, on='GEOID', how='right')
intersect_ssma_avg = gpd.GeoDataFrame(intersect_ssma_avg, geometry='geometry')

# reanme column to be interpretable
intersect_ssma_avg['ssma_mean'] = intersect_ssma_avg['mean']
intersect_ssma_avg.plot(column='ssma_mean')

In [ ]:
# drop unneeded columns
intersect_ssma_avg = intersect_ssma_avg[['GEOID', 'ssma_mean']]
intersect_ssma_avg

In [33]:
# save as a csv
intersect_ssma_avg.to_csv('natural_crop_condition_soil_anomaly_metric.csv', index=False)

### Step 4: Metadata

In [34]:
@append_metadata
def natural_soil_quality(input_csv, export=False, varname=''):    
    '''
    Calculates the average soil quality / crop condition values per California census tract.
    Metrics are:
    * Mean evaporative stress index
    * Mean surface soil moisture anomaly
    Data was sourced from: https://climateserv.servirglobal.net/map

    Methods
    -------
    Data columns were renamed, merged to 2022 census tract data, and averaged to attribute
    scores to all California tracts.
    
    Parameters
    ----------
    df: string
        the dataframe containing the crop condition data
    export: True/False boolean
        False = will not upload resulting df containing CAL CRAI crop condition metric to AWS
        True = will upload resulting df containing CAL CRAI crop condition metric to AWS
    import_csv: string
        name of the csv file to be uploaded to AWS

    Script
    ------
    natural_soil_quality.ipynb

    Note:
    This function assumes users have configured the AWS CLI such that their access key / secret key pair are
    stored in ~/.aws/credentials.
    See https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html for guidance.
    '''
    print('Data transformation: data converted to Cal-CRAI standardized coordinate reference system (CRS): 4269.')
    print('Data transformation: merge data to California tracts and calculate census tract average over time.')
    print('Data transformation: nan values are retained for each census tract without a soil quality metric(s), especially relevant for urban areas.')

    bucket_name = 'ca-climate-index'
    directory = '3_fair_data/index_data'
    export_filename = [input_csv]

    if export == True:
        upload_csv_aws(export_filename, bucket_name, directory)

    if export == False:
        print(f'{export_filename} uploaded to AWS.')

    if os.path.exists(input_csv):
        os.remove(input_csv)

In [35]:
input_csv = ['natural_crop_condition_esi_metric.csv',
            'natural_crop_condition_soil_anomaly_metric.csv']

varnames = [
    'natural_nasa_crop_esi',
    'natural_nasa_soil_moisture_anomaly']

for csv, var in zip(input_csv, varnames):
    natural_soil_quality(csv, export=False, varname=var)